In [213]:
import pandas as pd
import ast

In [214]:
import os

def find_repo_root(start_path):
    """
    useful general function for finding the (first, closest) repo root so github file paths work the same on different machines 
    """
    current_path = os.path.abspath(start_path)
    
    while True:
        # Check for the existence of the .git directory or other indicators
        if os.path.isdir(os.path.join(current_path, '.git')) or \
           os.path.isfile(os.path.join(current_path, 'README.md')):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        
        # Stop if we reach the root directory
        if parent_path == current_path:
            break
        
        current_path = parent_path

    return None  # Return None if not found

root = find_repo_root(os.getcwd())
print(root)

c:\Users\fitsl\Documents\Programming\UVM Programming Classes\PoCS\pocs_project


In [215]:
df = pd.read_csv(f"{root}/Data/letterboxd_search.csv", index_col=0)
df.columns
df['Tropes'].iloc[0]
df_old  = df.copy()

In [ ]:
df['nu_tropes'] = df['Tropes'].apply(lambda x: x.split(","))
# for trope in df[df['Movie'] == "Borat"]['nu_tropes']:
#     print(trope)

[' ADateWithRosiePalms', 'AccentuateTheNegative', 'AcceptableHardLuckTargets', 'ActionFilmQuietDramaScene', 'AdaptationExpansion', 'AliensStealCable', 'AlterEgoActing', 'AluminumChristmasTrees', 'AmbiguousInnocence', 'AmericansHateTingle', 'AndThereWasMuchRejoicing', 'AngryWhiteMan', 'ArtisticLicenceGeography', 'ArtisticLicenseGeography', 'AsLongAsItSoundsForeign', 'AsLongAsItsoundsForeign', 'AsLongAsitSoundsForeign', 'BackwardsR', 'BadHumorTruck', 'BadImpressionists', 'BadassMustache', 'BagOfKidnapping', 'BannedInBoston', 'BannedInChina', 'BannedinChina', 'BarelyThereSwimwear', 'BearsAreBadNews', 'BeautyInversion', 'BerserkButton', 'BerthaInTheAttic', 'BettyAndVeronica', 'BiLingualBonus', 'BigBeautifulWoman', 'BigDamnMovie', 'BigWhat', 'BilingualBonus', 'BlackComedyAnimalCruelty', 'BlackComedyRape', 'BlandNameProduct', 'BlatantLies', 'BlueAndOrangeMorality', 'BreadEggsMilkSquick', 'BrickJoke', 'BritishHumour', 'BrokenPedestal', 'BrooklynRage', 'BrotherSisterIncest', 'CMOTDibbler', 'Ca

In [ ]:
tropes = df['nu_tropes'].explode()
tropes = tropes.unique().tolist()
df_t = pd.DataFrame({'trope' : tropes})
df_t[df_t['trope'] == 'Not']


,trope
32643,Not


In [218]:
df['nu_tropes'].iloc[0]

[' ArgentineMedia',
 'BlackMagic',
 'ChekhovsGun',
 'ChekhovsGunman',
 'DramaticThunder',
 'DueToTheDead',
 'EvilMentor',
 'ExactWords',
 'FilmsOf20102014',
 'GrayRainOfDepression',
 'GreatOffscreenWar',
 'HesBack',
 'HistoricalHeroUpgrade',
 'HowWeGotHere',
 'ItAlwaysRainsAtFunerals',
 'OffscreenMomentOfAwesome',
 'RightfulKingReturns',
 'SexyDiscretionShot',
 'TheExile',
 'ThouShaltNotKill',
 'VeryLooselyBasedOnATrueStory',
 'WeWantOurJerkBack',
 'WhatsUpKingDude']

In [219]:
# def trope_grabber(trope, df):
#     mask = df['Tropes'].str.contains(trope, na=False)  # Boolean mask for matching tropes
#     movies = df[mask][['Movie', 'letterboxd_search']]  
#     return movies[['Movie', 'letterboxd_search']].values.tolist()  # Return list of tuples

# df_t['movie'] = df_t['trope'].apply(lambda trope: trope_grabber(trope, df))
# df_t

In [220]:
# df_t['movie'].iloc[0]

In [221]:
# df_t['Movie_Titles']  = df_t['movie'].apply(lambda x: [i[0] for i in x])
# df_t['letterboxd_searches'] = df_t['movie'].apply(lambda x: [i[1] for i in x])
# df_t.drop(columns=["movie"], inplace=True)
# df_t.to_csv(f"{root}/Data/Trope_storage/movies_by_trope_with_lettersearch.csv")

In [222]:
df = pd.read_csv(f"{root}/Data/Trope_storage/movies_by_trope_with_lettersearch.csv", index_col=0)
df['Movie_Titles'] = df['Movie_Titles'].apply(ast.literal_eval)
df['length'] = df['Movie_Titles'].apply(lambda x : len(x))
df.to_csv(f"{root}/Data/Trope_storage/movies_by_trope_with_lettersearch_and_length.csv")

In [223]:
for movie in df[df['trope'] == 'DueToTheDead']['Movie_Titles']:
    print(movie)

['Puerta  De  Hierro', 'U571', 'Black  Panther', 'Hardcore  Henry', 'Clerks', 'The  Quick  And  The  Dead', 'Skyfall', 'Serenity', 'Volver', 'Top  Gun  Maverick', 'Blade  Runner', 'Captain  Fantastic', 'Harry  Potter  And  The  Half  Blood  Prince', 'We  Were  Soldiers', 'La  Revolucion  Es  Un  Sueno  Eterno', 'Legend  Of  The  Black  Scorpion', 'Urban  Cowboy', 'Alles  Is  Liefde', 'Zombi2', 'Munich', 'The  Living  Wake', 'Red  Cliff', 'Indiana  Jones  And  The  Last  Crusade', 'North  Face', 'Son  Of  Saul', 'Duel  To  The  Death', 'Star  Trek  I  I  The  Wrath  Of  Khan', 'Let  Him  Go', 'Memphis  Belle', 'How  The  West  Was  Won', 'The  Phantom  Menace', 'The  Amityville  Horror', 'D  C911  Time  Of  Crisis', 'Pirates  Of  The  Caribbean  The  Curse  Of  The  Black  Pearl', 'The  Beastmaster', 'Contagion', 'Red  River', 'The  Transporter', 'Dracula  Prince  Of  Darkness', 'Anaconda', 'The  Abominable  Snowman', 'The  Man  Who  Would  Be  King', 'Red  Lion', 'The  Lobster', 'Subur

In [224]:
df.sort_values(by='length', ascending=False).head(50)


,trope,Movie_Titles,letterboxd_searches,length
19175,,"[Puerta De Hierro, Better Off Dead..., A ...",['https://letterboxd.com/search/Puerta+De+Hier...,14533
33945,Film,"[Puerta De Hierro, Better Off Dead..., A ...",['https://letterboxd.com/search/Puerta+De+Hier...,14306
20608,O,"[Puerta De Hierro, Better Off Dead..., A ...",['https://letterboxd.com/search/Puerta+De+Hier...,14296
30457,V,"[Puerta De Hierro, Better Off Dead..., A ...",['https://letterboxd.com/search/Puerta+De+Hier...,9274
35354,Go,"[Puerta De Hierro, Better Off Dead..., Tea...",['https://letterboxd.com/search/Puerta+De+Hier...,8213
4186,It,"[Puerta De Hierro, Better Off Dead..., A ...",['https://letterboxd.com/search/Puerta+De+Hier...,6341
32643,Not,"[Puerta De Hierro, Better Off Dead..., Tea...",['https://letterboxd.com/search/Puerta+De+Hier...,5732
32849,Hero,"[Puerta De Hierro, Team America World Pol...",['https://letterboxd.com/search/Puerta+De+Hier...,5291
37985,Mon,"[Better Off Dead..., Team America World P...",['https://letterboxd.com/search/Better+Off+Dea...,4972
34838,Death,"[Team America World Police, The Hills Hav...",['https://letterboxd.com/search/Team+America+W...,4924
